### RiskCalc examples.
Make sure that the working directory for this ipynb notebook is ./risktables/risktables

___
The class ```RiskCalcs``` performs basic portfolio risk calculations.  It returns a dictionary, whose items are either instances of Pandas ```DataFrame``` or floating point values representing portfolio aggregate risk measurements (VaR, Greeks, etc).
___

In [1]:
# include both the project package, and the project folder in sys.path
import sys,os
if  not './' in sys.path:
    sys.path.append(os.path.abspath('./'))
if  not '../' in sys.path:
    sys.path.append(os.path.abspath('../'))

from risktables import risk_tables
import pandas as pd
from IPython import display
import datetime

/Users/bperlman1/Documents/billybyte/pyliverisk/risktables/risktables/dgrid_components.py:17: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html
/Users/bperlman1/Documents/billybyte/pyliverisk/risktables/risktables/dgrid_components.py:18: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  import dash_core_components as dcc
/Users/bperlman1/Documents/billybyte/pyliverisk/risktables/risktables/dgrid_components.py:21: UserWarning: 
The dash_table package is deprecated. Please replace
`import dash_table` with `from dash import dash_table`

Also, if you're using any of the table format helpers (e.g. Group), replace 
`from dash_table.Format import Group` with 
`from dash.dash_table.Format import Group`
  import dash_table


#### Create an instance of RiskCalcs

In [2]:
# df_postgres_info = pd.read_csv('postgres_info.csv')
# display.display(df_postgres_info)
# prices_db_info = df_postgres_info.loc[df_postgres_info.config_name=='dashrisk_local']
rt = risk_tables.RiskCalcs(use_postgres=False)

#### Create an example portfolio consisting of hypothetical options on the S&P 500 Sector Spdr's

In [3]:
next_year = (datetime.datetime.now() + datetime.timedelta(weeks=52)).year
next_year = str(next_year)
df_spdr_options =  pd.read_csv('spdr_stocks.csv')
df_spdr_options.symbol = df_spdr_options.symbol.apply(lambda s: s.replace('2021',next_year))
display.display(df_spdr_options)


,symbol,position
0,XLB_20231231_83_c,120
1,XLC_20231231_52_p,192
2,XLE_20231231_90_c,111
3,XLF_20231231_36_p,278
4,XLI_20231231_102_c,98
5,XLK_20231231_131_p,76
6,XLP_20231231_75_c,133
7,XLU_20231231_71_p,141
8,XLV_20231231_135_c,74
9,XLY_20231231_140_p,71


#### Run risk calculations
*Market data must be fetched for each underlying, which causes the cell below to take about 10 seconds to run*

In [4]:
rt = risk_tables.RiskCalcs(use_postgres=False)
risk_dictionary = rt.calculate(df_spdr_options)

2023-01-24 09:05:12,822 - root - INFO - Start computing VaR 2023-01-24 09:05:12.822024


[*********************100%***********************]  1 of 1 completed
          open        high         low       close    volume  \
47  102.400002  102.540001  101.480003  101.589996   9274900   
48  101.940002  102.169998   99.620003   99.680000  15026000   
49   99.070000   99.320000   97.620003   97.629997  17155600   
50   97.709999   99.019997   97.309998   98.989998  12816500   
51   99.300003  100.610001   98.879997  100.070000  11112000   

                        date  
47 2023-01-17 00:00:00-05:00  
48 2023-01-18 00:00:00-05:00  
49 2023-01-19 00:00:00-05:00  
50 2023-01-20 00:00:00-05:00  
51 2023-01-23 00:00:00-05:00  
[*********************100%***********************]  1 of 1 completed
         open       high        low      close    volume  \
47  90.160004  91.080002  89.769997  90.139999  16966600   
48  90.559998  91.570000  88.400002  88.489998  19037600   
49  87.980003  89.949997  87.790001  89.589996  17146900   
50  89.779999  90.790001  88.870003  90.480003  142

/Users/bperlman1/Documents/billybyte/pyliverisk/risktables/risktables/var_models.py:253: FutureWarning:

The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.

/Users/bperlman1/Documents/billybyte/pyliverisk/risktables/risktables/var_models.py:255: FutureWarning:

The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.

/Users/bperlman1/Documents/billybyte/pyliverisk/risktables/risktables/var_models.py:257: FutureWarning:

The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be tr

#### List the names of the outputs

In [5]:
for k in risk_dictionary.keys():
    print(k)

yyyymmddhhmmssmmmmmm
df_std
df_atm_price
df_high_low
df_atm_info
df_risk_all
df_risk_by_underlying
df_var
port_var
sp_dollar_equiv
delta
gamma
vega
theta
df_hedge_ratios
df_corr
df_corr_price


#### Now show the actual values of each key

In [6]:
for k in risk_dictionary.keys():
    print(f'********************* {k} **************************')
    dict_item = risk_dictionary[k]
    object_to_print = dict_item
    if type(dict_item)==dict:
        if k[:2] == 'df':
            object_to_print = risk_tables.make_df(risk_dictionary[k])
    display.display(object_to_print)


********************* yyyymmddhhmmssmmmmmm **************************


'20230124090518246964'

********************* df_std **************************


,stdev,underlying
6,0.240495,XLB
4,0.276084,XLC
1,0.286160,XLE
2,0.202849,XLF
0,0.197312,XLI
8,0.325051,XLK
3,0.153255,XLP
5,0.204109,XLU
7,0.147273,XLV
9,0.293791,XLY


********************* df_atm_price **************************


...

""


********************* df_corr **************************


,*underlying,XLB,XLC,XLE,XLF,XLI,XLK,XLP,XLU,XLV,XLY
XLB,XLB,1.000000,0.839357,0.575051,0.900526,0.906616,0.879392,0.760055,0.746181,0.735015,0.825140
XLC,XLC,0.839357,1.000000,0.548870,0.824475,0.782500,0.877943,0.601936,0.572550,0.636344,0.845035
XLE,XLE,0.575051,0.548870,1.000000,0.570992,0.564202,0.573498,0.440645,0.311981,0.421635,0.542095
XLF,XLF,0.900526,0.824475,0.570992,1.000000,0.927196,0.866783,0.794512,0.781966,0.735923,0.870055
XLI,XLI,0.906616,0.782500,0.564202,0.927196,1.000000,0.866211,0.843897,0.793514,0.786432,0.809849
XLK,XLK,0.879392,0.877943,0.573498,0.866783,0.866211,1.000000,0.715882,0.738653,0.732889,0.915694
XLP,XLP,0.760055,0.601936,0.440645,0.794512,0.843897,0.715882,1.000000,0.814850,0.802728,0.655809
XLU,XLU,0.746181,0.572550,0.311981,0.781966,0.793514,0.738653,0.814850,1.000000,0.750109,0.687385
XLV,XLV,0.735015,0.636344,0.421635,0.735923,0.786432,0.732889,0.802728,0.750109,1.000000,0.654851
XLY,XLY,0.825140,0.845035,0.542095,0.870055,0.809849,0.915694,0.655809,0.687385,0.654851,1.000000


********************* df_corr_price **************************


,*underlying,XLB,XLC,XLE,XLF,XLI,XLK,XLP,XLU,XLV,XLY
XLB,XLB,1.000000,0.912630,0.245481,0.800602,0.883200,0.787195,0.387571,0.454249,0.388526,0.668810
XLC,XLC,0.912630,1.000000,0.334373,0.825629,0.788827,0.666734,0.166628,0.322011,0.187649,0.595275
XLE,XLE,0.245481,0.334373,1.000000,0.625713,0.166375,0.337580,-0.382018,-0.524485,-0.507465,0.532621
XLF,XLF,0.800602,0.825629,0.625713,1.000000,0.793313,0.653073,0.175302,0.123502,0.045217,0.709074
XLI,XLI,0.883200,0.788827,0.166375,0.793313,1.000000,0.621211,0.599935,0.640074,0.486621,0.480649
XLK,XLK,0.787195,0.666734,0.337580,0.653073,0.621211,1.000000,0.339677,0.131014,0.393427,0.899530
XLP,XLP,0.387571,0.166628,-0.382018,0.175302,0.599935,0.339677,1.000000,0.776003,0.873813,0.107356
XLU,XLU,0.454249,0.322011,-0.524485,0.123502,0.640074,0.131014,0.776003,1.000000,0.735272,-0.181369
XLV,XLV,0.388526,0.187649,-0.507465,0.045217,0.486621,0.393427,0.873813,0.735272,1.000000,0.139733
XLY,XLY,0.668810,0.595275,0.532621,0.709074,0.480649,0.899530,0.107356,-0.181369,0.139733,1.000000


In [9]:
pd.DataFrame(risk_dictionary['df_risk_all']).to_dict(orient="records")

[{'symbol': 'XLB_20231231_83_c',
  'underlying': 'XLB',
  'position': 120.0,
  'delta': 5972.665800177213,
  'gamma': 266.59160888051025,
  'vega': 3092.0138801775306,
  'theta': 116.77250258482557,
  'rho': 3989.4319356027045,
  'position_var': 225.23330682462284},
 {'symbol': 'XLC_20231231_52_p',
  'underlying': 'XLC',
  'position': 192.0,
  'delta': -3317.749881575041,
  'gamma': 394.5538783712502,
  'vega': 1936.4936100149532,
  'theta': 34.59242053154169,
  'rho': -1883.1992895099788,
  'position_var': -126.92102010046295},
 {'symbol': 'XLE_20231231_90_c',
  'underlying': 'XLE',
  'position': 111.0,
  'delta': 6161.3424223967195,
  'gamma': 244.30500257453733,
  'vega': 3372.065296249838,
  'theta': 128.47606147371673,
  'rho': 4478.615007752008,
  'position_var': 272.7247277868991},
 {'symbol': 'XLF_20231231_36_p',
  'underlying': 'XLF',
  'position': 278.0,
  'delta': -4145.26596733624,
  'gamma': 623.601091162825,
  'vega': 1340.9901803769392,
  'theta': 21.52532291874996,
  'r

In [ ]:
# Run this app with `python app.py` and
# visit http://127.0.0.1:8050/ in your web browser.

from dash import Dash,dash_table, html, dcc
import dash_bootstrap_components as dbc
import plotly.express as px
import pandas as pd

# external_scripts = [
#     {
#         'src': 'https://cdn.jsdelivr.net/npm/bootstrap@3.4.1/dist/js/bootstrap.min.js',
#         'integrity': 'sha384-aJ21OjlMXNL5UyIl/XNwTMqvzeRMZH2w8c5cRVpzpU8Y5bApTppSuUkhZXN0VxHd',
#         'crossorigin': 'anonymous'
#     },
# ]

# # external CSS stylesheets
# external_stylesheets = [
#     {
#         'href':"https://cdn.jsdelivr.net/npm/bootstrap@3.4.1/dist/css/bootstrap.min.css",
#         'rel': 'stylesheet',
#         'integrity': "sha384-HSMxcRTRxnN+Bdg0JdbxYKrThecOKuH5zCYotlSAcp1+c8xmyTe9GYg1l9a69psu" ,
#         'crossorigin': 'anonymous'
#     },
#     {
#         'href':"https://cdn.jsdelivr.net/npm/bootstrap@3.4.1/dist/css/bootstrap-theme.min.css",
#         'rel': 'stylesheet',
#         'integrity': "sha384-6pzBo3FDv/PJ8r2KRkGHifhEocL+1X2rVCTTkUfGk7/0pbek5mMa1upzvWbrUbOZ" ,
#         'crossorigin': 'anonymous'
#     },
    
# ]


# app = Dash(
#     __name__,
#     external_scripts=external_scripts,
#     external_stylesheets=external_stylesheets
# )
app = Dash(
    __name__,
)

# assume you have a "long-form" data frame
# see https://plotly.com/python/px-arguments/ for more options
# df = pd.DataFrame({
#     "Fruit": ["Apples", "Oranges", "Bananas", "Apples", "Oranges", "Bananas"],
#     "Amount": [4, 1, 2, 2, 4, 5],
#     "City": ["SF", "SF", "SF", "Montreal", "Montreal", "Montreal"]
# })

# fig = px.bar(df, x="Fruit", y="Amount", color="City", barmode="group")

df_risk_all = risk_tables.make_df(risk_dictionary['df_risk_all'])
dt = dash_table.DataTable(
    df_risk_all.to_dict('records'), 
    [
        {"name": i, "id": i} 
        for i in df_risk_all.columns
    ],
)
layout = html.Div(
    children=[
        html.Div(
#             style="width:98vw",
#             className="containerFluid",
            children=[
                html.H1(
                    children='Hello Dash',
                ),
                html.Div(
                    children='''
                    Dash: A web application framework for your data.
                    ''',
#                     className="row"
                ),
                html.Div(
#                     className="row",
                    children=[dt],
                ),
#                 dcc.Graph(
#                     id='example-graph',
#                     figure=fig
#                 ),        
            ],
        )
    
])

app.layout = layout
app.run_server(port=8999,debug=False,)

    